In [ ]:
!pip install -r requirements.txt

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:

from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
from openai import max_retries
llm = ChatOpenAI(
    model="o1",
    # temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)
# doesn't work with o1-mini and deepseek-r1.
# o1 was unable to format input for the calculator tool

# llm = ChatOllama(
#     # model="deepseek-r1",
#     model="llama3.2",
#     temperature=0,
#     max_retries=-1
#     # other params...
# )

In [77]:

response = llm.invoke("what is (4.5*2.1)^2.2?")
print(response)


content='First, compute the product inside the parentheses:\n\n4.5 × 2.1 = 9.45.\n\nNext, raise this result to the power of 2.2:\n\n(9.45)^(2.2).\n\nA good way to approximate this is using natural logarithms:\n\n(9.45)^(2.2) = exp(2.2 × ln(9.45)).\n\nSince ln(9.45) is approximately 2.246, we get  \n2.2 × 2.246 = 4.9412,  \nand  \nexp(4.9412) is roughly 139.8–140.0.\n\nHence, (4.5 × 2.1)^(2.2) ≈ 140.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 4923, 'prompt_tokens': 21, 'total_tokens': 4944, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 4736, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'o1-2024-12-17', 'system_fingerprint': 'fp_d91c966719', 'finish_reason': 'stop', 'logprobs': None} id='run-fe0cf48f-e74b-45ef-985d-edce46d2d0b5-0' usage_metadata={'input_tokens': 21, 'output_tokens': 4923, 'total_tokens': 4944, 'inp

In [78]:
from langchain.chains import LLMMathChain # Replace with your actual LLM class
from langchain.agents import Tool

llm_math = LLMMathChain.from_llm(llm=llm)

# initialize the math tool
math_tool = Tool(
    name='Calculator',
    func=llm_math.run,
    description='Useful for when you need to answer questions about math.'
)
# when giving tools to LLM, we must pass as list of tools
tools = [math_tool]

In [79]:
tools[0].name, tools[0].description

('Calculator', 'Useful for when you need to answer questions about math.')

In [80]:
from langchain.agents import load_tools

tools = load_tools(
    ['llm-math'],
    llm=llm
)

In [81]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3
)

In [ ]:
zero_shot_agent("what is (4.5*2.1)^2.2?")